In [1]:
%run theme.ipynb

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.offline as pyo
from collections import Counter

# Read the CSV file
df = pd.read_csv('f1_winners_detailed_analysis.csv')

# Define circuit order based on the ranking
circuit_order = [
    'China', 'Miami', 'Bahrain', 'Sakhir', 'South Korea', 'Qatar', 'Azerbaijan',
    'Brazil', 'Eifel', 'Abu Dhabi', 'India', 'USA', 'Belgium', 'Turkey', 'Russia',
    'Styria', 'Malaysia', 'Germany', 'Austria', 'Singapore'
]

# Filter to only include circuits that exist in our data
available_circuits = df['Grand_Prix'].unique()
circuit_order_filtered = [circuit for circuit in circuit_order if circuit in available_circuits]

# Add any remaining circuits not in the specified order
remaining_circuits = [circuit for circuit in available_circuits if circuit not in circuit_order_filtered]
final_circuit_order = circuit_order_filtered + sorted(remaining_circuits)
# Count occurrences of each starting position category by circuit
data_for_chart = []

for circuit in final_circuit_order:
    circuit_data = df[df['Grand_Prix'] == circuit]
    category_counts = circuit_data['Starting_Position_Category'].value_counts()
    
    p1_count = category_counts.get('P1', 0)
    p2_count = category_counts.get('P2', 0)
    p3_count = category_counts.get('P3', 0)
    outside_count = category_counts.get('Outside Top 3', 0)
    
    data_for_chart.append({
        'Circuit': circuit,
        'P1': p1_count,
        'P2': p2_count,
        'P3': p3_count,
        'Outside Top 3': outside_count,
        'Total': p1_count + p2_count + p3_count + outside_count
    })

chart_df = pd.DataFrame(data_for_chart)

# Define colors for each starting position category
colors = {
    'P1': "#A80000",           # Dark red
    'P2': "#FF3333",           # Lighter red
    'P3': "#FF7E7E",           # Even lighter red
    'Outside Top 3': "#D4D4D4"  # Gray
}

# Create the stacked bar chart
fig = go.Figure()

# Add bars for each category
for category in ['P1', 'P2', 'P3', 'Outside Top 3']:
    fig.add_trace(go.Bar(
        name=category,
        x=chart_df['Circuit'],
        y=chart_df[category],
        marker_color=colors[category],
        text=chart_df[category],
        textposition='inside',
        textfont=dict(color='white', size=10)
    ))

# Update layout
fig.update_layout(
    title={
        'text': 'Win conversie van startposities in de Formule 1 van de top 20 circuits met gemiddeld de meeste overtakes',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 16, 'color': "#000000"}
    },
    xaxis={
        'title': 'Grand Prix Circuit',
        'tickangle': -45,
        'title_font': {'size': 14, 'color': "#000000"},
        'tickfont': {'size': 11, 'color': "#000000"}
    },
    yaxis={
        'title': 'Number of Wins',
        'title_font': {'size': 14, 'color': "#000000"},
        'tickfont': {'size': 12, 'color': "#000000"}
    },
    barmode='stack',
    plot_bgcolor='white',
    paper_bgcolor='white',
    width=1000,
    height=600,
    margin=dict(l=60, r=30, t=80, b=120),
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='center',
        x=0.5
    ),
    font=dict(family="Arial, sans-serif")
)

# Show the chart
fig.show()

# Also save as HTML for embedding in Jupyter Book
fig.write_html("f1_starting_positions_bar_chart.html")

category_totals = df['Starting_Position_Category'].value_counts()
for category in ['P1', 'P2', 'P3', 'Outside Top 3']:
    count = category_totals.get(category, 0)
    percentage = (count / len(df)) * 100
    print(f"{category}: {count} wins ({percentage:.1f}%)")

P1: 109 wins (47.6%)
P2: 59 wins (25.8%)
P3: 26 wins (11.4%)
Outside Top 3: 35 wins (15.3%)
